<a href="https://colab.research.google.com/github/jeaneigsi/Humain-/blob/main/Debertav3_lsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://alt.qcri.org/semeval2014/task4/data/uploads/semeval14-absa-base-eval-valid.zip
!unzip semeval14-absa-base-eval-valid.zip
!wget https://github.com/kmayutrisna/SemEval-Dataset/blob/main/SemEval%202014%20Task%204/Restaurants_Train.xml
!wget https://github.com/kmayutrisna/SemEval-Dataset/blob/main/SemEval%202014%20Task%204/Laptops_Train.csv
!wget https://github.com/kmayutrisna/SemEval-Dataset/blob/main/SemEval%202014%20Task%204/Laptops_Train.csv

In [14]:
!python SemEval2014.py

  File "/content/SemEval2014.py", line 38
    <link rel="canonical" href="https://huggingface.co/datasets/Yaxin/SemEval2014Task4Raw/blob/main/SemEval2014.py">  <!-- HEAD_svelte-1oal594_START --><style>.blob-line-num::before {
                                                                                                                                       ^
SyntaxError: invalid decimal literal


In [4]:
!wget https://github.com/kmayutrisna/SemEval-Dataset/blob/main/SemEval%202014%20Task%204/Laptops_Train.csv

--2024-09-06 09:25:21--  https://github.com/kmayutrisna/SemEval-Dataset/blob/main/SemEval%202014%20Task%204/Laptops_Train.csv
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘Laptops_Train.csv’

Laptops_Train.csv       [ <=>                ]   1.20M  --.-KB/s    in 0.09s   

2024-09-06 09:25:22 (13.7 MB/s) - ‘Laptops_Train.csv’ saved [1255035]



In [ ]:
!pip install datasets

# Prepocessing

In [30]:
import pandas as pd
import numpy as np
from datasets import load_dataset



In [31]:
Restaurants_Train=load_dataset('alexcadillon/SemEval2014Task4','restaurants')
Laptops_Train=load_dataset('alexcadillon/SemEval2014Task4','laptops')

In [32]:
Restaurants_df=Restaurants_Train['train'].to_pandas()
laptops_df=Laptops_Train['train'].to_pandas()

In [33]:
#check correct
Restaurants_df.isna().sum()
laptops_df.isna().sum()

,0
sentenceId,0
text,0
aspectTerms,0


# Deberta-base LSA Report

We will eval it on [here](https://github.com/kmayutrisna/SemEval-Dataset)

In [34]:
from transformers import AutoTokenizer, AutoModel

tokenizer=AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")
model=AutoModel.from_pretrained("microsoft/deberta-v3-base")

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

* Normal inference

In [36]:
input='The service was good , but other is really bad'
inputs=tokenizer(input, return_tensors='pt', padding=True,truncation=True,max_length=512)
inputs

{'input_ids': tensor([[  1, 279, 506, 284, 397, 366, 304, 340, 269, 431, 966,   2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [15]:
tokenizer.decode(inputs['input_ids'][0])

'[CLS] The service was good, but other is really bad[SEP]'

In [37]:
import torch
with torch.no_grad():
  model(**inputs)
  outputs=model(**inputs)

In [38]:
outputs

BaseModelOutput(last_hidden_state=tensor([[[ 0.1389,  0.2316,  0.0410,  ..., -0.0660,  0.1466, -0.0018],
         [ 0.1486, -0.1391,  0.1828,  ..., -1.0466,  0.1544, -0.1429],
         [ 0.2820,  0.5533,  0.1707,  ..., -0.9439, -0.0039, -0.0498],
         ...,
         [ 0.1630,  0.2054,  0.1206,  ..., -0.0283,  0.0965,  0.0287],
         [ 0.3194,  0.6675,  0.1107,  ...,  0.1889, -0.4547,  0.3727],
         [ 0.2203,  0.2504, -0.0049,  ..., -0.0574,  0.1638,  0.0258]]]), hidden_states=None, attentions=None)

In [39]:
embeddings=outputs.last_hidden_state
embeddings.shape

torch.Size([1, 12, 768])

**Tips** : The squeeze() function in PyTorch (or NumPy) is used to remove dimensions of size 1 from a tensor.
Before: `embeddings.shape = (1, 512, 768) (batch size 1, 512 tokens, 768 dimensions).`   
After: `embeddings.squeeze(0) gives embeddings.shape = (512, 768) (removing the batch size dimension).`

* Appy LSa on last layer

In [25]:
from sklearn.decomposition import TruncatedSVD
import torch
import numpy as np
from transformers import Trainer, TrainingArguments
from datasets import load_metric
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix, precision_score

In [43]:
def get_embeddings(text):
  inputs=tokenizer(text, return_tensors='pt', padding=True,truncation=True,max_length=512)
  # inference
  with torch.no_grad():
    output=model(**inputs)
    hidden_states=output.last_hidden_state
 # apply lsa
  embeddings=hidden_states.squeeze(0).numpy()
  svd=TruncatedSVD(n_components=100)
  reduced_embeddings=svd.fit_transform(embeddings)

  return reduced_embeddings


In [46]:
get_embeddings(" i love so much")

array([[ 6.40322971e+00, -1.04383526e+01,  1.46081886e+01,
        -3.60971284e+00, -8.32375288e-01,  6.01719260e-01],
       [ 2.82256145e+01,  2.25611649e+01,  8.18492699e+00,
        -4.71714878e+00, -3.39641595e+00, -2.10821629e-03],
       [ 3.81039734e+01, -8.05090666e-01,  4.67973328e+00,
         1.45498085e+01,  2.89446163e+00,  6.34288788e-03],
       [ 3.24927979e+01, -2.32613754e+00, -5.38651752e+00,
        -8.83478355e+00,  9.36377716e+00, -3.49891186e-03],
       [ 3.82714615e+01, -1.03871765e+01, -1.10041733e+01,
        -2.34408164e+00, -8.05614662e+00, -3.75723839e-03],
       [ 6.30585289e+00, -1.04935961e+01,  1.47938461e+01,
        -3.38868546e+00, -7.97591925e-01, -5.99076629e-01]], dtype=float32)

In [ ]:

def compute_metrics(eval_pred):
  labels=eval_pred.label_ids
  pred=np.argmax(eval_pred.predictions,axis=1)
  accuracy=accuracy_score(labels,pred)
  recall=recall_score(labels,pred,average='weighted')
  f1=f1_score(labels,pred,average='weighted')
  precision=precision_score(labels,pred,average='weighted')

  return {'accuracy :' accuracy, 'recall :' recall, 'f1_score :': f1, 'precision :': precision}

TrainingArguments= {
    'output_dir':'./results',
    'num-train_epochs':3,
    'per_device_train_batch_size':8,
    'per_device_eval_batch_size':8,
    'warmup_steps':500,
    'weight_decay':0.01,
    'logging_dir':'./logs
}

trainer= Trainer{
    model=get_embeddings,
    args=TrainingArguments,
    train_dataset=Restaurants_df,
    eval_dataset=laptops_df,
    compute_metrics=compute_metrics
}

In [36]:
clean = embeddings.squeeze(0).numpy()
clean.shape

(12, 768)

# **Instruct ABASE ATE inference**
(https://huggingface.co/liujiawu/Llama-3-8b-review-absa-4bit-v0.1/tree/main)



In [17]:
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM

tokenizer=AutoTokenizer.from_pretrained("kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-combined")
model=AutoModelForSeq2SeqLM.from_pretrained("kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-combined")

In [23]:
inputs=tokenizer("the food was amazing", return_tensors='pt', padding=True,truncation=True,max_length=512)

In [30]:
inputs

{'input_ids': tensor([[   8,  542,   47, 1237,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [31]:
model.generate(inputs['input_ids'])


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[   0,   11,    8,  313,   47, 1287,    5,   37,  542,   47, 1237,   11,
            8,  313,   47, 1287,    5,    1]])

In [35]:
output=tokenizer.batch_decode(model.generate(inputs['input_ids']), skip_special_tokens=True)

In [36]:
output

['and the service was excellent. The food was amazing and the service was excellent.']

In [25]:
tokenizer.decode(inputs['input_ids'][0])

'the food was amazing</s>'

In [24]:
inputs

{'input_ids': tensor([[   8,  542,   47, 1237,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [19]:
inputs=tokenizer("The service was really good", return_tensors='pt', padding=True,truncation=True,max_length=512)
inputs

{'input_ids': tensor([[ 37, 313,  47, 310, 207,   1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [37]:
!git clone https://github.com/kevinscaria/InstructABSA.git
%cd InstructABSA
!pip install -r requirements.txt

Cloning into 'InstructABSA'...
remote: Enumerating objects: 562, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 562 (delta 123), reused 197 (delta 106), pack-reused 338 (from 1)
Receiving objects: 100% (562/562), 1.58 MiB | 11.11 MiB/s, done.
Resolving deltas: 100% (320/320), done.
/content/InstructABSA
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled p

In [46]:
%cd /content/InstructABSA


/content/InstructABSA


In [44]:
!git clone https://huggingface.co/kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-combined



Cloning into 'ate_tk-instruct-base-def-pos-neg-neut-combined'...
remote: Enumerating objects: 23, done.
remote: Total 23 (delta 0), reused 0 (delta 0), pack-reused 23 (from 1)
Unpacking objects: 100% (23/23), 607.16 KiB | 5.14 MiB/s, done.


In [47]:
!python run_model.py -mode cli -task ate \
-model_checkpoint kevinscaria/ate_tk-instruct-base-def-pos-neg-neut-combined \
-test_input 'The cab ride was amazing but the service was pricey'

2024-09-06 18:32:37.021412: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-06 18:32:37.398238: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-06 18:32:37.495353: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-06 18:32:38.100946: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-06 18:32:40.970666: W tensorflow/compiler/tf2

# DBERTA

In [48]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-large-absa-v1.1")

model = AutoModelForSequenceClassification.from_pretrained("yangheng/deberta-v3-large-absa-v1.1")


tokenizer_config.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

In [49]:
inputs=tokenizer("The service was really good", return_tensors='pt', padding=True,truncation=True,max_length=512)


In [51]:
with torch.no_grad():
  outputs=model(**inputs)



In [62]:
# Example sentence and aspect
sentence = "The battery life of this phone is fantastic, but the camera is disappointing."
aspect = "camera"  # You focus on the sentiment towards the camera aspect.

In [63]:
input_text = f"{aspect} [SEP] {sentence}"

# Tokenize the input with both the aspect and the sentence
inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)

# Run inference to get logits
with torch.no_grad():
    outputs = model(**inputs)

# Extract logits
logits = outputs.logits

# Convert logits to probabilities
probs = torch.softmax(logits, dim=-1)

# Get the predicted sentiment label (e.g., positive, neutral, negative)
predicted_label = torch.argmax(probs, dim=-1).item()

# Print the result
print(f"Aspect: {aspect}")
print(f"Predicted sentiment label: {predicted_label}")
print(f"Probabilities: {probs}")

Aspect: camera
Predicted sentiment label: 0
Probabilities: tensor([[0.9704, 0.0261, 0.0035]])
